# LSTM Model


In [3]:
# Install necessary libraries
!pip install tensorflow pandas numpy scikit-learn matplotlib

# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split # Not used directly for final split, but good practice
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
import matplotlib.pyplot as plt
from google.colab import drive
from tensorflow.keras import backend as K # For clearing session

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# --- Configuration ---
DATA_PATH = '/content/drive/My Drive/Non-Thesis Project/Tetouan City Power Consumption.csv'
TARGET_MONTHS = [4, 7, 10, 12]  # Months for training and testing
TRAIN_DAYS = range(1, 8)     # Days 1-7 of the target months for training
# *** MODIFIED: Test days range changed to 8-14 ***
TEST_DAYS = range(8, 15)    # Days 8-14 of the target months for testing
# ---------------------------------------------------
TOLERANCE_PERCENTAGE = 15.0

# --- Define Fixed LSTM Hyperparameters (for the 3-layer model) ---
FIXED_LSTM_UNITS = 192
FIXED_DENSE_UNITS = 128
FIXED_DROPOUT = 0.25
FIXED_LEARNING_RATE = 0.001
# ----------------------------------------------------------------

# --- Data Loading and Preprocessing ---
print("Loading and preprocessing data...")
# Added error handling for file loading
try:
    data = pd.read_csv(DATA_PATH)
except FileNotFoundError:
    print(f"ERROR: Data file not found at {DATA_PATH}")
    raise
data.columns = [col.strip().replace(' ', '_').lower() for col in data.columns]
zone_columns = [col for col in data.columns if 'power_consumption' in col]
print("Detected zone columns:", zone_columns)
data['datetime'] = pd.to_datetime(data['datetime'])

# Resample the data to hourly intervals FIRST
numeric_cols = data.select_dtypes(include=np.number).columns.tolist()
data = data.set_index('datetime')
hourly_data = data[numeric_cols].resample('H').mean()
hourly_data.reset_index(inplace=True) # Keep datetime as column temporarily

# Create/Recreate temporal features AFTER resampling
hourly_data['hour'] = hourly_data['datetime'].dt.hour
hourly_data['dayofweek'] = hourly_data['datetime'].dt.dayofweek
hourly_data['month'] = hourly_data['datetime'].dt.month
hourly_data['dayofmonth'] = hourly_data['datetime'].dt.day # Renamed from 'day' for clarity
hourly_data['dayofyear'] = hourly_data['datetime'].dt.dayofyear
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
hourly_data['day_name'] = hourly_data['dayofweek'].apply(lambda x: day_names[x])
month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
hourly_data['month_name'] = hourly_data['month'].apply(lambda x: month_names[x-1])

# Handle potential NaNs from resampling (only for non-target numeric cols)
numeric_features_to_fill = [col for col in hourly_data.select_dtypes(include=np.number).columns if col not in zone_columns]
for col in numeric_features_to_fill:
    if hourly_data[col].isnull().any():
        print(f"Filling NaNs in '{col}' using ffill/bfill/mean...")
        hourly_data[col].fillna(method='ffill', inplace=True)
        hourly_data[col].fillna(method='bfill', inplace=True)
        if hourly_data[col].isnull().any():
            mean_val = hourly_data[col].mean()
            print(f"Warning: Filling remaining NaNs in '{col}' with overall mean ({mean_val:.2f}).")
            hourly_data[col].fillna(mean_val, inplace=True)

# Add lag features
print("Adding lag features...")
lags_to_add = list(range(1, 7)) # Use a list for clarity
for lag in lags_to_add:
    for zone_col in zone_columns:
        if zone_col in hourly_data.columns: hourly_data[f'{zone_col}_lag{lag}'] = hourly_data[zone_col].shift(lag)

# Drop rows with NaN values (mainly due to lagging)
print(f"Dropping initial rows with NaNs from lags...")
cols_with_initial_nans = [f'{zc}_lag{lag}' for zc in zone_columns for lag in lags_to_add if f'{zc}_lag{lag}' in hourly_data.columns]
original_len = len(hourly_data)
hourly_data.dropna(subset=cols_with_initial_nans, inplace=True)
print(f"Dropped {original_len - len(hourly_data)} rows.")
if len(hourly_data) == 0: raise ValueError("Not enough data remaining after dropping initial NaN rows from lags.")

print(f"Data shape after preprocessing: {hourly_data.shape}")

# --- Filtering and Feature Preparation ---
print(f"Filtering and preparing features for TRAIN_DAYS={list(TRAIN_DAYS)} and TEST_DAYS={list(TEST_DAYS)}...")
# Set index AFTER preprocessing and BEFORE filtering
hourly_data.set_index('datetime', inplace=True)

# Filter using the defined day ranges
# *** CORRECTED: Use .day instead of .dayofmonth for DatetimeIndex ***
train_data = hourly_data[(hourly_data.index.month.isin(TARGET_MONTHS)) & (hourly_data.index.day.isin(TRAIN_DAYS))].copy()
test_data = hourly_data[(hourly_data.index.month.isin(TARGET_MONTHS)) & (hourly_data.index.day.isin(TEST_DAYS))].copy()
# -------------------------------------------------------------------


if train_data.empty: raise ValueError("Train data is empty after filtering. Check TARGET_MONTHS/TRAIN_DAYS.")
if test_data.empty: raise ValueError(f"Test data is empty after filtering. Check TARGET_MONTHS/TEST_DAYS (currently {list(TEST_DAYS)}).")

print(f"Train shape: {train_data.shape}, Test shape: {test_data.shape}")

# Prepare features (keep only numeric features for this basic model)
train_features = train_data[[col for col in train_data.columns if col not in zone_columns]]
test_features = test_data[[col for col in test_data.columns if col not in zone_columns]]
# Drop the string name columns if they exist
train_features = train_features.drop(columns=['day_name', 'month_name'], errors='ignore')
test_features = test_features.drop(columns=['day_name', 'month_name'], errors='ignore')

# Select only numeric features now for scaling
train_features_numeric = train_features.select_dtypes(include=np.number)
test_features_numeric = test_features.select_dtypes(include=np.number)

# Extract targets
train_targets = train_data[zone_columns]
test_targets = test_data[zone_columns]

# Align columns based on numeric features selected
train_cols = train_features_numeric.columns.tolist()
test_cols = test_features_numeric.columns.tolist()

print(f"Initial number of numeric features in train: {len(train_cols)}")
print(f"Initial number of numeric features in test: {len(test_cols)}")

# Align columns - Crucial step
missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    print(f"Adding missing column '{c}' to test set (filled with 0).")
    test_features_numeric[c] = 0

extra_in_test = set(test_cols) - set(train_cols)
if extra_in_test:
    print(f"Dropping extra columns from test set: {extra_in_test}")
    test_features_numeric = test_features_numeric.drop(columns=list(extra_in_test))

# Final alignment
try:
    test_features_numeric = test_features_numeric[train_cols] # Reorder test columns to match train
    print(f"Final number of features in train set: {len(train_features_numeric.columns)}")
    print(f"Final number of features in test set (aligned): {len(test_features_numeric.columns)}")
    # print("Features:", train_cols) # Uncomment to see list of features
except KeyError as e:
    print(f"ERROR: Column alignment failed. Missing columns in test set after adding: {e}")
    print("Train columns:", train_cols)
    print("Test columns before alignment:", test_cols)
    raise

# Check for NaN/inf values after preparation
if train_features_numeric.isnull().values.any() or np.isinf(train_features_numeric.values).any():
    print("ERROR: NaN or Inf found in training features AFTER preparation. Check preprocessing.")
    print("Columns with NaNs/Infs in Training Features:\n", train_features_numeric.isin([np.nan, np.inf, -np.inf]).sum())
    raise ValueError("NaN/Inf in training features.")
if test_features_numeric.isnull().values.any() or np.isinf(test_features_numeric.values).any():
    print("ERROR: NaN or Inf found in test features AFTER preparation. Check preprocessing.")
    print("Columns with NaNs/Infs in Test Features:\n", test_features_numeric.isin([np.nan, np.inf, -np.inf]).sum())
    raise ValueError("NaN/Inf in test features.")


# --- Scaling and Reshaping ---
print("Scaling and reshaping data...")
scaler = MinMaxScaler() # Feature scaler
target_scaler = StandardScaler() # Target scaler

train_features_scaled = scaler.fit_transform(train_features_numeric)
test_features_scaled = scaler.transform(test_features_numeric) # Use transform only on test

train_targets_scaled = target_scaler.fit_transform(train_targets)
test_targets_scaled = target_scaler.transform(test_targets) # Use transform only on test

# Reshape for LSTM [samples, timesteps, features] - assuming 1 timestep
train_features_reshaped = train_features_scaled.reshape((train_features_scaled.shape[0], 1, train_features_scaled.shape[1]))
test_features_reshaped = test_features_scaled.reshape((test_features_scaled.shape[0], 1, test_features_scaled.shape[1]))

print(f"Train Features Reshaped: {train_features_reshaped.shape}")
print(f"Test Features Reshaped: {test_features_reshaped.shape}")
print(f"Train Targets Scaled Shape: {train_targets_scaled.shape}")
print(f"Test Targets Scaled Shape: {test_targets_scaled.shape}")


# --- Model Definition (Modified for 3 Layers) ---
def build_lstm_model_3layer(lstm_units, dense_units, dropout_rate, lr, n_features, n_outputs):
    """
    Builds a 3-layer Bidirectional LSTM model for the basic feature set.
    """
    model = Sequential([
        # Layer 1
        Bidirectional(LSTM(lstm_units, return_sequences=True, input_shape=(1, n_features))),
        Dropout(dropout_rate),
        BatchNormalization(),
        # Layer 2
        Bidirectional(LSTM(lstm_units, return_sequences=True)),
        Dropout(dropout_rate),
        BatchNormalization(),
        # Layer 3
        Bidirectional(LSTM(lstm_units, return_sequences=False)),
        Dropout(dropout_rate),
        BatchNormalization(),
        # Dense Layers
        Dense(dense_units, activation='relu'),
        Dense(n_outputs) # Linear activation for regression
    ])
    optimizer = Adam(learning_rate=lr)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

# --- Model Building and Training (Using Fixed Parameters - 3 Layers) ---
print("\nBuilding and training 3-Layer LSTM model (Basic Features)...")
n_features = train_features_reshaped.shape[2]
n_outputs = train_targets_scaled.shape[1]
K.clear_session() # Clear previous models from memory

model = build_lstm_model_3layer(
    FIXED_LSTM_UNITS,
    FIXED_DENSE_UNITS,
    FIXED_DROPOUT,
    FIXED_LEARNING_RATE,
    n_features,
    n_outputs
)

print("Using Fixed Hyperparameters:")
print(f"  LSTM Units (x3 Layers): {FIXED_LSTM_UNITS}")
print(f"  Dense Units: {FIXED_DENSE_UNITS}")
print(f"  Dropout Rate: {FIXED_DROPOUT:.3f}")
print(f"  Learning Rate: {FIXED_LEARNING_RATE:.6f}")

model.summary()

# --- Training Setup (Callbacks and Fit) ---
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=1e-6, verbose=1)

history = model.fit(
    train_features_reshaped,
    train_targets_scaled,
    epochs=150,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Plot Training Loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss (Basic Features, 3x{FIXED_LSTM_UNITS} LSTM)') # Updated title
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
# Adjust ylim for better visualization
if len(history.history['loss']) > 5:
    max_loss_val = max(history.history['loss'][5:] + history.history['val_loss'][5:])
    min_loss_val = min(history.history['loss'] + history.history['val_loss'])
    plt.ylim(bottom=max(0, min_loss_val * 0.9), top=max_loss_val * 1.1)
elif len(history.history['loss']) > 0:
    max_loss_val = max(history.history['loss'] + history.history['val_loss'])
    min_loss_val = min(history.history['loss'] + history.history['val_loss'])
    plt.ylim(bottom=max(0, min_loss_val * 0.9), top=max_loss_val * 1.1)
else:
    plt.ylim(bottom=0)
plt.show()

if hasattr(early_stopping, 'stopped_epoch') and early_stopping.stopped_epoch > 0 and early_stopping.restore_best_weights:
    print(f"Training stopped early at epoch {early_stopping.stopped_epoch + 1}. Best weights restored.")
elif hasattr(early_stopping, 'stopped_epoch') and early_stopping.stopped_epoch > 0:
     print(f"Training stopped early at epoch {early_stopping.stopped_epoch + 1}, but best weights might not have been restored.")
else:
    print("Training completed all epochs or early stopping did not trigger.")


# --- Evaluation ---
print("\nEvaluating model on the test set...")
test_predictions_scaled = model.predict(test_features_reshaped)

# Inverse transform predictions and actuals
actual_test_targets = target_scaler.inverse_transform(test_targets_scaled)
predicted_test_targets = target_scaler.inverse_transform(test_predictions_scaled)

# Clip negative predictions
predicted_test_targets[predicted_test_targets < 0] = 0

# --- Create plotting dataframe ---
plot_ready = False
# Check length using the index of test_data and the length of predictions
if len(test_data.index) == len(actual_test_targets):
    # Create DataFrames for actual and predicted values using the test data index
    actual_df = pd.DataFrame(actual_test_targets, index=test_data.index, columns=zone_columns)
    pred_df = pd.DataFrame(predicted_test_targets, index=test_data.index, columns=[f"{zone}_predicted" for zone in zone_columns])

    # Combine index, actuals, and predictions into a new DataFrame for plotting
    plot_test_df = pd.concat([actual_df, pred_df], axis=1)
    # Re-add time components needed for plotting titles/labels directly from the index
    plot_test_df['hour'] = plot_test_df.index.hour
    plot_test_df['day'] = plot_test_df.index.day # Use .day
    plot_test_df['month_name'] = plot_test_df.index.strftime('%B')
    plot_test_df['day_name'] = plot_test_df.index.day_name()

    plot_ready = True
    print(f"Plotting dataframe created successfully.")
else:
    print(f"Error: Length mismatch! Original Test DF index length {len(test_data.index)} vs Predictions length {len(actual_test_targets)}. Skipping plotting.")
    plot_test_df = None # Ensure it's None if not created


# --- Plotting Results ---
if plot_ready and plot_test_df is not None:
    print(f"Plotting daily forecasts (Basic Features, 3x{FIXED_LSTM_UNITS} LSTM) for days {list(TEST_DAYS)}...") # Updated description
    # Get unique month names present in the test data index
    unique_month_names = plot_test_df['month_name'].unique()

    for zone in zone_columns:
        zone_name_plot = zone.replace("_power_consumption", "").replace("zone_", "Zone ").capitalize()

        for month_name in unique_month_names:
            month_data = plot_test_df[plot_test_df['month_name'] == month_name]
            if month_data.empty: continue

            available_days_in_plot_df = month_data['day'].unique()
            # Ensure plot_days only includes days defined in TEST_DAYS that are actually present in the filtered test data
            plot_days = sorted([d for d in TEST_DAYS if d in available_days_in_plot_df])

            for day in plot_days:
                # Filter data specifically for this day within the month
                filtered_data = month_data[month_data['day'] == day]
                if filtered_data.empty: continue # Should not happen if day is in plot_days, but safe check

                actual_col_name = zone
                predicted_col_name = f"{zone}_predicted"
                if actual_col_name not in filtered_data.columns or predicted_col_name not in filtered_data.columns:
                    print(f"Warning: Column missing for {zone}, {month_name}, {day}. Skipping plot.")
                    continue

                actual_day = filtered_data[actual_col_name]
                predicted_day = filtered_data[predicted_col_name]

                # Ensure 1D input for metrics
                if not isinstance(actual_day, pd.Series) or not isinstance(predicted_day, pd.Series): continue
                if actual_day.ndim > 1 or predicted_day.ndim > 1: continue

                # Calculate metrics safely
                try:
                    daily_rmse = sqrt(mean_squared_error(actual_day, predicted_day))
                    daily_mae = mean_absolute_error(actual_day, predicted_day)
                    daily_r2 = r2_score(actual_day, predicted_day)
                except ValueError as e_met:
                    print(f"Warning: Metric calculation error for {zone}, {month_name}, {day}: {e_met}")
                    daily_rmse, daily_mae, daily_r2 = np.nan, np.nan, np.nan
                except Exception as e_gen:
                    print(f"Unexpected error calculating metrics for {zone}, {month_name}, {day}: {e_gen}")
                    daily_rmse, daily_mae, daily_r2 = np.nan, np.nan, np.nan

                plt.figure(figsize=(14, 7))
                bar_width = 0.35
                x = np.arange(len(filtered_data)) # Should be 24 hours
                plt.bar(x - bar_width/2, actual_day, width=bar_width, label='Actual', color='black', alpha=0.8)
                plt.bar(x + bar_width/2, predicted_day, width=bar_width, label='Predicted', color='orange', alpha=0.8)

                day_name_str = filtered_data['day_name'].iloc[0] if not filtered_data.empty else "UnknownDay"
                plt.title(f"{zone_name_plot} Load Forecast (Basic, 3x{FIXED_LSTM_UNITS} LSTM) - {day_name_str}, {month_name} {day}", fontsize=14) # Updated title
                plt.xlabel('Hour of Day', fontsize=12)
                plt.ylabel('Power Consumption', fontsize=12)
                plt.xticks(x, filtered_data['hour'], rotation=45, ha='right')
                plt.legend(fontsize=10)
                plt.grid(axis='y', linestyle='--', alpha=0.7)
                plt.tight_layout(rect=[0, 0, 0.85, 1]) # Adjust layout

                stats_text = (f"Daily Metrics:\n"
                              f"RMSE: {daily_rmse:.2f}\n"
                              f"MAE:  {daily_mae:.2f}\n"
                              f"R²:   {daily_r2:.3f}")
                plt.text(1.02, 0.95, stats_text, transform=plt.gca().transAxes, fontsize=10,
                         verticalalignment='top', bbox=dict(boxstyle='round,pad=0.5', facecolor='orange', alpha=0.6))
                plt.show()
else:
    print("Skipping plots due to data length mismatch or plotting dataframe creation failed.")

# --- Final Evaluation Metrics ---
print(f"\n--- Test Set Evaluation Metrics (Basic Features, 3x{FIXED_LSTM_UNITS} LSTM - Per Zone) ---") # Updated description
all_actuals_flat = []
all_predictions_flat = []

for i, zone in enumerate(zone_columns):
    zone_name_print = zone.replace("zone_", "Zone_").replace("_power_consumption", "_")
    actual = actual_test_targets[:, i]
    predicted = predicted_test_targets[:, i]
    all_actuals_flat.extend(actual)
    all_predictions_flat.extend(predicted)

    rmse = sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    try: r2 = r2_score(actual, predicted)
    except ValueError: r2 = np.nan

    print(f"\n--- {zone_name_print} ---")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAE: {mae:.2f}")
    print(f"  R² Score: {r2:.4f}")

print(f"\n--- Overall Model Evaluation (Basic Features, 3x{FIXED_LSTM_UNITS} LSTM - All Zones Combined) ---") # Updated description
all_actuals_flat = np.array(all_actuals_flat)
all_predictions_flat = np.array(all_predictions_flat)

overall_rmse = sqrt(mean_squared_error(all_actuals_flat, all_predictions_flat))
overall_mae = mean_absolute_error(all_actuals_flat, all_predictions_flat)
try: overall_r2 = r2_score(all_actuals_flat, all_predictions_flat)
except ValueError: overall_r2 = np.nan

# Calculate Tolerance Accuracy Safely
abs_error = np.abs(all_actuals_flat - all_predictions_flat)
non_zero_actuals = all_actuals_flat[all_actuals_flat > 1e-6]
mean_nonzero_actual = np.mean(non_zero_actuals) if len(non_zero_actuals) > 0 else 1.0

absolute_min_tolerance = 10 # Use the same minimum absolute tolerance
percentage_tolerance_value = (TOLERANCE_PERCENTAGE / 100.0) * np.abs(all_actuals_flat)
tolerance_threshold = np.maximum(percentage_tolerance_value, absolute_min_tolerance)

is_within_tolerance = abs_error <= tolerance_threshold
tolerance_accuracy = np.mean(is_within_tolerance) * 100

print(f"  Overall RMSE: {overall_rmse:.2f}")
print(f"  Overall MAE: {overall_mae:.2f}")
print(f"  Overall R² Score: {overall_r2:.4f}")
print(f"\n  Tolerance-Based Accuracy ({TOLERANCE_PERCENTAGE}%): {tolerance_accuracy:.2f}%")
print(f"     (Accuracy defined as % of predictions within +/- {TOLERANCE_PERCENTAGE}% of actual, with min absolute tolerance of {absolute_min_tolerance} units)")
print("----------------------------------------------------------")

print("\nProcessing complete.")

Output hidden; open in https://colab.research.google.com to view.

#Hybrid Fuzzy-LSTM Model

In [2]:
# Install necessary libraries
!pip install tensorflow scikit-fuzzy scikit-learn pandas numpy matplotlib

# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split # Keep for final validation split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt, pi # Removed log10, floor as they weren't used after opt removal
import matplotlib.pyplot as plt
import skfuzzy as fuzz
from skfuzzy import control as ctrl
# --- FCM Import ---
from skfuzzy.cluster import cmeans, cmeans_predict
# ------------------
from google.colab import drive
import random
import warnings
# TensorFlow/Keras
from tensorflow.keras import backend as K # For clearing session

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# --- Configuration ---
DATA_PATH = '/content/drive/My Drive/Non-Thesis Project/Tetouan City Power Consumption.csv'
TARGET_MONTHS = [4, 7, 10, 12]
TRAIN_DAYS = range(1, 8)      # Days 1-7 for training
# *** MODIFIED: Test days range changed to 8-14 ***
TEST_DAYS = range(8, 15)     # Days 8-14 for testing
# ---------------------------------------------------
FUZZY_POWER_MAX = 50000
TEMP_MIN = 0
TEMP_MAX = 45
TOLERANCE_PERCENTAGE = 15.0
ROLLING_WINDOWS = [6, 24]

# --- FCM Configuration ---
FCM_N_CLUSTERS = 7 # Number of fuzzy clusters to identify (Hyperparameter)
FCM_M = 2.0        # Fuzziness parameter (Standard default)
FCM_ERROR = 0.005  # Stopping criterion error
FCM_MAXITER = 1000 # Max iterations for FCM training

# --- Define Fixed LSTM Hyperparameters ---
FIXED_LSTM_UNITS = 192 # Units for EACH of the 3 LSTM layers
FIXED_DENSE_UNITS = 128
FIXED_DROPOUT = 0.25
FIXED_LEARNING_RATE = 0.001
# --------------------------------------

# --- Fuzzy Logic Setup (For Feature Engineering - Keep Enhanced Version) ---
# 1. Hour
hour_universe = np.arange(0, 24, 1); hour_fuzzy = ctrl.Antecedent(hour_universe, 'hour_fuzzy')
hour_fuzzy['night'] = fuzz.trapmf(hour_fuzzy.universe, [0, 0, 4, 6]); hour_fuzzy['morning'] = fuzz.trimf(hour_fuzzy.universe, [5, 9, 12]); hour_fuzzy['afternoon'] = fuzz.trimf(hour_fuzzy.universe, [11, 15, 18]); hour_fuzzy['evening'] = fuzz.trapmf(hour_fuzzy.universe, [17, 20, 23, 23])
# 2. Lag1 Power
fuzzy_power_lag1 = {}
def setup_fuzzy_power_lag1(zone_cols, power_max_val):
    global fuzzy_power_lag1; power_min = 0; power_max = power_max_val; power_step = max(1, (power_max - power_min) / 100); power_universe = np.arange(power_min, power_max + power_step, power_step)
    for zone in zone_cols:
        var_name = f"{zone}_lag1_fuzzy"; power_lag1_fuzzy = ctrl.Antecedent(power_universe, var_name)
        low_thresh = power_max * 0.3; med_thresh_1 = power_max * 0.25; med_thresh_2 = power_max * 0.6; high_thresh = power_max * 0.55
        power_lag1_fuzzy['low'] = fuzz.trapmf(power_universe, [power_min, power_min, low_thresh*0.8, low_thresh]); power_lag1_fuzzy['medium'] = fuzz.trimf(power_universe, [med_thresh_1, (med_thresh_1+med_thresh_2)/2 , med_thresh_2]); power_lag1_fuzzy['high'] = fuzz.trapmf(power_universe, [high_thresh, high_thresh*1.1, power_max, power_max])
        fuzzy_power_lag1[zone] = power_lag1_fuzzy
# 3. Day Type
daytype_universe = np.arange(0, 7, 1); day_type_fuzzy = ctrl.Antecedent(daytype_universe, 'day_type_fuzzy')
day_type_fuzzy['weekday'] = fuzz.trapmf(daytype_universe, [0, 0, 4, 4.5]); day_type_fuzzy['weekend'] = fuzz.trapmf(daytype_universe, [4.5, 5, 6, 6])
# 4. Season
month_universe_season = np.arange(1, 13, 1); season_fuzzy = ctrl.Antecedent(month_universe_season, 'season_fuzzy')
season_fuzzy['spring'] = fuzz.trimf(month_universe_season, [3, 4, 5])
season_fuzzy['summer'] = fuzz.trimf(month_universe_season, [6, 7, 8])
season_fuzzy['autumn'] = fuzz.trimf(month_universe_season, [9, 10, 11])
season_fuzzy['winter'] = fuzz.trimf(month_universe_season, [11, 12, 13]) # Corrected previously
# 5. Temperature
temp_universe = np.arange(TEMP_MIN, TEMP_MAX + 1, 1); temp_fuzzy = ctrl.Antecedent(temp_universe, 'temp_fuzzy')
temp_fuzzy['cold'] = fuzz.trapmf(temp_universe, [TEMP_MIN, TEMP_MIN, 5, 12]); temp_fuzzy['mild'] = fuzz.trimf(temp_universe, [8, 15, 22]); temp_fuzzy['warm'] = fuzz.trimf(temp_universe, [18, 25, 32]); temp_fuzzy['hot'] = fuzz.trapmf(temp_universe, [28, 35, TEMP_MAX, TEMP_MAX])

# --- Enhanced Fuzzification Function (Includes DayType, Season, Temp) ---
def fuzzify_data_enhanced(row, zone_cols):
    memberships = {}
    # Hour
    hour_val = row.get('hour_original', np.nan)
    if pd.notna(hour_val): memberships['hour_night'] = fuzz.interp_membership(hour_universe, hour_fuzzy['night'].mf, hour_val); memberships['hour_morning'] = fuzz.interp_membership(hour_universe, hour_fuzzy['morning'].mf, hour_val); memberships['hour_afternoon'] = fuzz.interp_membership(hour_universe, hour_fuzzy['afternoon'].mf, hour_val); memberships['hour_evening'] = fuzz.interp_membership(hour_universe, hour_fuzzy['evening'].mf, hour_val)
    else: memberships['hour_night'] = 0.0; memberships['hour_morning'] = 0.0; memberships['hour_afternoon'] = 0.0; memberships['hour_evening'] = 0.0
    # Lag1 Power
    for zone in zone_cols:
        lag1_col = f'{zone}_lag1'; power_val_lag1 = row.get(lag1_col, np.nan)
        if zone in fuzzy_power_lag1:
            fuzz_var = fuzzy_power_lag1[zone]; power_univ = fuzz_var.universe
            if pd.notna(power_val_lag1): power_val_lag1_clipped = np.clip(power_val_lag1, power_univ.min(), power_univ.max()); memberships[f'{zone}_lag1_low'] = fuzz.interp_membership(power_univ, fuzz_var['low'].mf, power_val_lag1_clipped); memberships[f'{zone}_lag1_medium'] = fuzz.interp_membership(power_univ, fuzz_var['medium'].mf, power_val_lag1_clipped); memberships[f'{zone}_lag1_high'] = fuzz.interp_membership(power_univ, fuzz_var['high'].mf, power_val_lag1_clipped)
            else: memberships[f'{zone}_lag1_low'] = 0.0; memberships[f'{zone}_lag1_medium'] = 0.0; memberships[f'{zone}_lag1_high'] = 0.0
        else: memberships[f'{zone}_lag1_low'] = 0.0; memberships[f'{zone}_lag1_medium'] = 0.0; memberships[f'{zone}_lag1_high'] = 0.0
    # Day Type
    dayofweek_val = row.get('dayofweek', np.nan)
    if pd.notna(dayofweek_val): memberships['day_weekday'] = fuzz.interp_membership(daytype_universe, day_type_fuzzy['weekday'].mf, dayofweek_val); memberships['day_weekend'] = fuzz.interp_membership(daytype_universe, day_type_fuzzy['weekend'].mf, dayofweek_val)
    else: memberships['day_weekday'] = 0.0; memberships['day_weekend'] = 0.0
    # Season
    month_val = row.get('month', np.nan)
    try:
        if pd.notna(month_val):
            memberships['season_spring'] = fuzz.interp_membership(month_universe_season, season_fuzzy['spring'].mf, month_val)
            memberships['season_summer'] = fuzz.interp_membership(month_universe_season, season_fuzzy['summer'].mf, month_val)
            memberships['season_autumn'] = fuzz.interp_membership(month_universe_season, season_fuzzy['autumn'].mf, month_val)
            memberships['season_winter'] = fuzz.interp_membership(month_universe_season, season_fuzzy['winter'].mf, month_val)
        else:
            memberships['season_spring'], memberships['season_summer'], memberships['season_autumn'], memberships['season_winter'] = 0.0, 0.0, 0.0, 0.0
    except ValueError:
        print(f"Warning: Could not calculate seasonal membership for month {month_val}")
        memberships['season_spring'], memberships['season_summer'], memberships['season_autumn'], memberships['season_winter'] = 0.0, 0.0, 0.0, 0.0

    # Temperature
    temp_val = row.get('temperature', np.nan)
    if pd.notna(temp_val): temp_val_clipped = np.clip(temp_val, temp_universe.min(), temp_universe.max()); memberships['temp_cold'] = fuzz.interp_membership(temp_universe, temp_fuzzy['cold'].mf, temp_val_clipped); memberships['temp_mild'] = fuzz.interp_membership(temp_universe, temp_fuzzy['mild'].mf, temp_val_clipped); memberships['temp_warm'] = fuzz.interp_membership(temp_universe, temp_fuzzy['warm'].mf, temp_val_clipped); memberships['temp_hot'] = fuzz.interp_membership(temp_universe, temp_fuzzy['hot'].mf, temp_val_clipped)
    else: memberships['temp_cold'] = 0.0; memberships['temp_mild'] = 0.0; memberships['temp_warm'] = 0.0; memberships['temp_hot'] = 0.0
    return pd.Series(memberships)

# --- Data Loading and Preprocessing (with FCM + Enhanced Fuzzy) ---
print("Loading and preprocessing data...")
# Added error handling for file loading
try:
    data = pd.read_csv(DATA_PATH)
except FileNotFoundError:
    print(f"ERROR: Data file not found at {DATA_PATH}")
    raise
data.columns = [col.strip().replace(' ', '_').lower() for col in data.columns]; zone_columns = [col for col in data.columns if 'power_consumption' in col]; data['datetime'] = pd.to_datetime(data['datetime'])
setup_fuzzy_power_lag1(zone_columns, FUZZY_POWER_MAX)
numeric_cols = data.select_dtypes(include=np.number).columns.tolist(); data = data.set_index('datetime'); hourly_data = data[numeric_cols].resample('H').mean(); hourly_data.reset_index(inplace=True)

print("Engineering base features (time, weather)...")
hourly_data['hour_original'] = hourly_data['datetime'].dt.hour; hourly_data['dayofweek'] = hourly_data['datetime'].dt.dayofweek; hourly_data['month'] = hourly_data['datetime'].dt.month; hourly_data['dayofyear'] = hourly_data['datetime'].dt.dayofyear; hourly_data['day'] = hourly_data['datetime'].dt.day # Use .day here
weather_cols = ['temperature', 'humidity', 'wind_speed', 'general_diffuse_flows', 'diffuse_flows']
for col in weather_cols:
    if col in hourly_data.columns:
        hourly_data[col].fillna(method='ffill', inplace=True)
        hourly_data[col].fillna(method='bfill', inplace=True)
        if hourly_data[col].isnull().any():
            mean_val = hourly_data[col].mean()
            print(f"Warning: Filling remaining NaNs in '{col}' with overall mean ({mean_val:.2f}).")
            hourly_data[col].fillna(mean_val, inplace=True)
    else:
        print(f"Warning: Weather column '{col}' not found in data.")

# Updated holiday definition for clarity
holidays_2017 = {
    1: [1],
     12: [1] # Add more if known
}
hourly_data['is_holiday'] = 0
for month, days in holidays_2017.items():
    hourly_data.loc[(hourly_data['month'] == month) & (hourly_data['day'].isin(days)), 'is_holiday'] = 1
print(f"Identified {hourly_data['is_holiday'].sum()} holiday hours based on defined list.")

print("Engineering cyclical time features...")
hourly_data['hour_sin'] = np.sin(2 * pi * hourly_data['hour_original'] / 24.0); hourly_data['hour_cos'] = np.cos(2 * pi * hourly_data['hour_original'] / 24.0); hourly_data['dayofweek_sin'] = np.sin(2 * pi * hourly_data['dayofweek'] / 7.0); hourly_data['dayofweek_cos'] = np.cos(2 * pi * hourly_data['dayofweek'] / 7.0); hourly_data['month_sin'] = np.sin(2 * pi * hourly_data['month'] / 12.0); hourly_data['month_cos'] = np.cos(2 * pi * hourly_data['month'] / 12.0); hourly_data['dayofyear_sin'] = np.sin(2 * pi * hourly_data['dayofyear'] / 365.25); hourly_data['dayofyear_cos'] = np.cos(2 * pi * hourly_data['dayofyear'] / 365.25)

print(f"Adding rolling mean features (windows: {ROLLING_WINDOWS})...");
for zone_col in zone_columns:
    if zone_col in hourly_data.columns:
      for window in ROLLING_WINDOWS: hourly_data[f'{zone_col}_roll_mean_{window}h'] = hourly_data[zone_col].rolling(window=window, min_periods=1).mean().shift(1)
    else:
        print(f"Warning: Zone column '{zone_col}' not found for rolling mean calculation.")

print("Adding lag features (including daily and weekly)..."); lags_to_add = list(range(1, 7)) + [24, 168]
for lag in lags_to_add:
    for zone_col in zone_columns:
        if zone_col in hourly_data.columns: hourly_data[f'{zone_col}_lag{lag}'] = hourly_data[zone_col].shift(lag)
        else:
             print(f"Warning: Zone column '{zone_col}' not found for lag {lag} calculation.")


cols_with_initial_nans = []; cols_with_initial_nans.extend([f'{zc}_lag{lag}' for zc in zone_columns for lag in lags_to_add if f'{zc}_lag{lag}' in hourly_data.columns]); cols_with_initial_nans.extend([f'{zc}_roll_mean_{w}h' for zc in zone_columns for w in ROLLING_WINDOWS if f'{zc}_roll_mean_{w}h' in hourly_data.columns])
print(f"Dropping initial rows with NaNs from lags/rolling features..."); original_len = len(hourly_data); hourly_data.dropna(subset=cols_with_initial_nans, inplace=True); print(f"Dropped {original_len - len(hourly_data)} rows.");
if len(hourly_data) == 0: raise ValueError("Not enough data remaining after dropping initial NaN rows.")

# --- Fuzzy C-Means Clustering ---
print("\n--- Applying Fuzzy C-Means Clustering ---")
fcm_base_zone = 'zone_1_power_consumption' if 'zone_1_power_consumption' in zone_columns else zone_columns[0]
print(f"Using '{fcm_base_zone}' lags as representative for FCM.")

fcm_feature_cols = [
    f'{fcm_base_zone}_lag1', f'{fcm_base_zone}_lag24', f'{fcm_base_zone}_lag168',
    'temperature', 'hour_sin', 'hour_cos', 'dayofweek_sin', 'dayofweek_cos'
]
fcm_feature_cols = [col for col in fcm_feature_cols if col in hourly_data.columns]
fcm_membership_cols = []
if len(fcm_feature_cols) < 3:
    print(f"Warning: Only {len(fcm_feature_cols)} suitable features found for FCM clustering: {fcm_feature_cols}. Skipping FCM.")
else:
    print(f"Features selected for FCM: {fcm_feature_cols}")
    if hourly_data[fcm_feature_cols].isnull().values.any():
        print("Warning: NaNs detected in FCM feature columns. Attempting to fill with column means.")
        for col in fcm_feature_cols:
             if hourly_data[col].isnull().any():
                 mean_val = hourly_data[col].mean()
                 hourly_data[col].fillna(mean_val, inplace=True)
                 print(f"Filled NaNs in '{col}' with mean {mean_val:.2f}")
        if hourly_data[fcm_feature_cols].isnull().values.any():
             print("ERROR: NaNs still present in FCM features after filling. Skipping FCM.")
             fcm_data = None
        else:
             fcm_data = hourly_data[fcm_feature_cols].values.T
    else:
        fcm_data = hourly_data[fcm_feature_cols].values.T

    if 'fcm_data' in locals() and fcm_data is not None:
        # Ensure datetime index exists for filtering BEFORE FCM training filter
        if 'datetime' in hourly_data.columns:
            hourly_data['datetime'] = pd.to_datetime(hourly_data['datetime'])
            # *** CORRECTED HERE: Set drop=True (or remove it as it's default) ***
            hourly_data.set_index('datetime', inplace=True, drop=True)
        else:
            # If datetime column doesn't exist, maybe index is already datetime? Check.
             if not isinstance(hourly_data.index, pd.DatetimeIndex):
                  raise ValueError("Cannot set datetime index for FCM filtering - 'datetime' column missing and index is not datetime.")
             # If index is already datetime, we are good to proceed

        # Ensure index is DatetimeIndex before filtering
        if not isinstance(hourly_data.index, pd.DatetimeIndex):
             raise TypeError("Index must be DatetimeIndex for filtering in FCM section.")

        train_indices_dt = hourly_data.index[hourly_data.index.month.isin(TARGET_MONTHS) & hourly_data.index.day.isin(TRAIN_DAYS)]
        # Use iloc with get_indexer for integer positions
        train_loc_indices = hourly_data.index.get_indexer(train_indices_dt)
        train_loc_indices = train_loc_indices[train_loc_indices != -1] # Remove -1 if index mismatch occurred

        if len(train_loc_indices) > FCM_N_CLUSTERS:
            fcm_train_data_subset = fcm_data[:, train_loc_indices]

            fcm_scaler = StandardScaler()
            fcm_train_data_scaled_subset = fcm_scaler.fit_transform(fcm_train_data_subset.T).T
            fcm_data_scaled = fcm_scaler.transform(fcm_data.T).T

            print(f"Training FCM with {FCM_N_CLUSTERS} clusters on {fcm_train_data_scaled_subset.shape[1]} samples...")
            try:
                cntr, u_train, _, _, _, _, _ = cmeans(
                    fcm_train_data_scaled_subset, FCM_N_CLUSTERS, FCM_M, error=FCM_ERROR, maxiter=FCM_MAXITER, init=None, seed=42
                )
                print("FCM training complete.")

                print("Predicting FCM memberships for all data...")
                u_all, _, _, _, _, _ = cmeans_predict(
                    fcm_data_scaled, cntr, FCM_M, error=FCM_ERROR, maxiter=FCM_MAXITER, seed=42
                )

                fcm_membership_cols = [f'fcm_cluster_{i}' for i in range(FCM_N_CLUSTERS)]
                # Use original hourly_data index (which should be datetime) for the DataFrame
                fcm_membership_df = pd.DataFrame(u_all.T, index=hourly_data.index, columns=fcm_membership_cols)
                # Join back to hourly_data
                hourly_data = hourly_data.join(fcm_membership_df)
                print(f"Added {FCM_N_CLUSTERS} FCM membership features.")
                if hourly_data[fcm_membership_cols].isnull().values.any():
                    print("Warning: NaNs detected in FCM membership columns after adding. Filling with 0.")
                    hourly_data[fcm_membership_cols] = hourly_data[fcm_membership_cols].fillna(0)

            except Exception as e:
                print(f"Error during FCM processing: {e}. Skipping FCM features.")
                fcm_membership_cols = []
        else:
            print(f"Warning: Not enough training data found for FCM ({len(train_loc_indices)} samples) based on filtering. Need > {FCM_N_CLUSTERS}. Skipping FCM.")
            fcm_membership_cols = []
# --- End FCM ---

# --- Apply Standard Fuzzification (Hour, Lag1, DayType, Season, Temp) ---
print("\nApplying standard fuzzification (Hour, Lag1 Power, DayType, Season, Temp)...")
cols_for_fuzzify_base = ['hour_original', 'dayofweek', 'month', 'temperature']
cols_for_fuzzify_lags = [f'{zc}_lag1' for zc in zone_columns if f'{zc}_lag1' in hourly_data.columns]
cols_for_fuzzify = [col for col in cols_for_fuzzify_base if col in hourly_data.columns] + cols_for_fuzzify_lags

if not all(c in hourly_data.columns for c in cols_for_fuzzify_base):
     print("Warning: Not all base columns for fuzzification found. Skipping fuzzification for missing ones.")
if not cols_for_fuzzify_lags:
     print("Warning: No Lag1 columns found for fuzzification.")

if cols_for_fuzzify:
    # Apply fuzzification using the DataFrame index to ensure alignment
    # Pass the DataFrame itself to apply, it will handle rows
    fuzzy_features_df = hourly_data[cols_for_fuzzify].apply(lambda row: fuzzify_data_enhanced(row, zone_columns), axis=1)
    # Explicitly set the index of the new DataFrame to match hourly_data
    fuzzy_features_df.index = hourly_data.index
    # Concatenate standard fuzzy features using join to align on index
    hourly_data = hourly_data.join(fuzzy_features_df, rsuffix='_fuzzy_result') # Add suffix if potential overlap
    # Check for NaNs introduced by fuzzification join
    fuzzy_cols_added = fuzzy_features_df.columns
    if hourly_data[fuzzy_cols_added].isnull().values.any():
        print("Warning: NaNs detected in standard fuzzy features after adding. Filling with 0.")
        hourly_data[fuzzy_cols_added] = hourly_data[fuzzy_cols_added].fillna(0)

else:
    print("Skipping standard fuzzification as required columns are missing.")
# --- End Standard Fuzzification ---

# Final check for NaNs before splitting
if hourly_data.isnull().values.any():
    print("Warning: NaNs detected in DataFrame before filtering. Dropping rows with any NaNs.")
    initial_rows = len(hourly_data)
    # Identify columns with NaNs for debugging
    nan_cols_before_drop = hourly_data.columns[hourly_data.isnull().any()].tolist()
    print(f"Columns with NaNs before drop: {nan_cols_before_drop}")
    hourly_data.dropna(inplace=True)
    print(f"Dropped {initial_rows - len(hourly_data)} rows due to NaNs.")

if len(hourly_data) == 0: raise ValueError("Data is empty after final NaN drop before filtering.")

# Ensure datetime index is still set after potential drops/joins
if not isinstance(hourly_data.index, pd.DatetimeIndex):
    # If the index was lost (e.g., by joining), try to set it again
    if 'datetime' in hourly_data.columns:
        print("Resetting datetime index...")
        hourly_data['datetime'] = pd.to_datetime(hourly_data['datetime'])
        hourly_data.set_index('datetime', inplace=True)
    # If index is not datetime AND datetime column doesn't exist, error
    else:
         # Check if maybe reset_index happened implicitly and 'datetime' exists in columns
         if 'datetime' in hourly_data.columns:
              hourly_data.set_index('datetime', inplace=True)
              print("Set datetime index from existing column.")
         else:
              # Check if level_0 might be the index
              if 'level_0' in hourly_data.columns and isinstance(hourly_data['level_0'].iloc[0], pd.Timestamp):
                   print("Using 'level_0' as datetime index.")
                   hourly_data.rename(columns={'level_0':'datetime'}, inplace=True)
                   hourly_data.set_index('datetime', inplace=True)
              else:
                   raise ValueError("Datetime index lost and cannot be reliably reset before filtering.")


print(f"Data shape after ALL preprocessing, FCM & Enhanced Fuzzy: {hourly_data.shape}")


# --- Filtering Data ---
print(f"\nFiltering data for final train/test subsets (TRAIN_DAYS={list(TRAIN_DAYS)}, TEST_DAYS={list(TEST_DAYS)})...")
# Datetime index should already be set correctly here
full_train_data = hourly_data[(hourly_data.index.month.isin(TARGET_MONTHS)) & (hourly_data.index.day.isin(TRAIN_DAYS))].copy()
full_test_data = hourly_data[(hourly_data.index.month.isin(TARGET_MONTHS)) & (hourly_data.index.day.isin(TEST_DAYS))].copy()

if full_train_data.empty: raise ValueError("Full Train data is empty after filtering. Check TARGET_MONTHS/TRAIN_DAYS and data availability.")
if full_test_data.empty: raise ValueError(f"Full Test data is empty after filtering. Check TARGET_MONTHS/TEST_DAYS (currently {list(TEST_DAYS)}) and data availability.")

print(f"Full Train shape: {full_train_data.shape}, Full Test shape: {full_test_data.shape}")


# --- Feature Preparation Function (Includes FCM + Enhanced Fuzzy) ---
def prepare_features_targets(train_df, test_df=None):
    print("Preparing features and targets (FCM + Enhanced Fuzzy)...")
    # Exclude targets AND the original columns used for fuzzification/cyclical/clustering
    # 'datetime' column is created by reset_index, so it should be excluded here
    base_originals = ['hour_original', 'dayofweek', 'month', 'dayofyear', 'day', 'temperature', 'datetime']
    lag1_originals = [f'{zc}_lag1' for zc in zone_columns]
    fcm_originals_to_exclude = list(set(fcm_feature_cols)) # Unique FCM features used

    # Also exclude original weather columns if they exist and are not needed directly
    weather_originals = [col for col in weather_cols if col != 'temperature'] # Temp is in base_originals

    features_to_exclude = zone_columns + base_originals + lag1_originals + fcm_originals_to_exclude + weather_originals
    features_to_exclude = sorted(list(set(features_to_exclude))) # Unique sorted list

    # Ensure exclusion list only contains columns actually present in the train_df
    features_to_exclude_present_train = [col for col in features_to_exclude if col in train_df.columns]
    print(f"Columns being excluded as features: {features_to_exclude_present_train}")

    train_features = train_df.drop(columns=features_to_exclude_present_train, errors='ignore')
    train_targets = train_df[zone_columns]
    train_features_encoded = train_features # No explicit encoding needed here
    train_cols = train_features_encoded.columns.tolist()
    print(f"Number of final features in training set: {len(train_cols)}")
    # print("Final Training Features:", train_cols) # Uncomment to debug features

    test_features_encoded = None; test_targets = None
    if test_df is not None:
        # Ensure test_df has the same columns available before dropping
        features_to_exclude_present_test = [col for col in features_to_exclude if col in test_df.columns]
        test_features = test_df.drop(columns=features_to_exclude_present_test, errors='ignore')
        test_targets = test_df[zone_columns]
        test_features_encoded = test_features
        test_cols = test_features_encoded.columns.tolist()
        print(f"Initial number of features in test set: {len(test_cols)}")

        # Align columns - Crucial step
        missing_in_test = set(train_cols) - set(test_cols)
        for c in missing_in_test:
            print(f"Adding missing column '{c}' to test set (filled with 0).")
            test_features_encoded[c] = 0

        extra_in_test = set(test_cols) - set(train_cols)
        if extra_in_test:
            print(f"Dropping extra columns from test set: {extra_in_test}")
            test_features_encoded = test_features_encoded.drop(columns=list(extra_in_test))

        # Final alignment
        try:
            test_features_encoded = test_features_encoded[train_cols] # Reorder test columns to match train
            print(f"Final number of features in test set (aligned): {len(test_features_encoded.columns)}")
        except KeyError as e:
            print(f"ERROR: Column alignment failed. Missing columns in test set after adding: {e}")
            print("Train columns:", train_cols)
            print("Test columns before alignment:", test_cols)
            raise


    # Check for NaN/inf values after preparation
    if train_features_encoded.isnull().values.any() or np.isinf(train_features_encoded.values).any():
        print("ERROR: NaN or Inf found in training features AFTER preparation. Check feature engineering steps.")
        print("Columns with NaNs/Infs in Training Features:\n", train_features_encoded.isin([np.nan, np.inf, -np.inf]).sum())
        raise ValueError("NaN/Inf in training features.")
    if test_features_encoded is not None and (test_features_encoded.isnull().values.any() or np.isinf(test_features_encoded.values).any()):
        print("ERROR: NaN or Inf found in test features AFTER preparation. Check feature engineering steps.")
        print("Columns with NaNs/Infs in Test Features:\n", test_features_encoded.isin([np.nan, np.inf, -np.inf]).sum())
        raise ValueError("NaN/Inf in test features.")


    return train_features_encoded, train_targets, test_features_encoded, test_targets

# --- Prepare data ---
# *** Use reset_index() here as intended, the previous fix should prevent the error ***
final_train_features_enc, final_train_targets, final_test_features_enc, final_test_targets = prepare_features_targets(
    full_train_data.reset_index(), full_test_data.reset_index()
)

# --- Scaling ---
print("Scaling data (MinMaxScaler for features, StandardScaler for targets)...")
# Fit scalers ONLY on the final training set
final_scaler_feat = MinMaxScaler(); final_scaler_targ = StandardScaler()
final_train_features_scaled = final_scaler_feat.fit_transform(final_train_features_enc)
final_test_features_scaled = final_scaler_feat.transform(final_test_features_enc)
final_train_targets_scaled = final_scaler_targ.fit_transform(final_train_targets)
# Scale test targets using the same scaler fitted on train targets for inverse transform later
final_test_targets_scaled = final_scaler_targ.transform(final_test_targets)


# --- Reshaping ---
def reshape_data(features, targets=None):
    # Reshape features to (samples, timesteps, features) - Here timesteps=1
    features_reshaped = features.reshape((features.shape[0], 1, features.shape[1]))
    # Targets remain 2D (samples, num_zones)
    return features_reshaped, targets

final_train_feat_res, final_train_targ_res = reshape_data(final_train_features_scaled, final_train_targets_scaled)
final_test_feat_res, final_test_targ_res = reshape_data(final_test_features_scaled, final_test_targets_scaled) # Reshape test targets too

print(f"Final Train Features Shape: {final_train_feat_res.shape}, Targets Shape: {final_train_targ_res.shape}")
print(f"Final Test Features Shape: {final_test_feat_res.shape}, Targets Shape: {final_test_targ_res.shape}")


# --- LSTM Model Building Function (Modified for 3 Layers) ---
def build_lstm_model(lstm_units, dense_units, dropout_rate, lr, n_features, n_outputs):
    """ Builds a 3-layer Bidirectional LSTM model. """
    model = Sequential([
        Bidirectional(LSTM(lstm_units, return_sequences=True, input_shape=(1, n_features))),
        Dropout(dropout_rate), BatchNormalization(),
        Bidirectional(LSTM(lstm_units, return_sequences=True)),
        Dropout(dropout_rate), BatchNormalization(),
        Bidirectional(LSTM(lstm_units, return_sequences=False)),
        Dropout(dropout_rate), BatchNormalization(),
        Dense(dense_units, activation='relu'),
        Dense(n_outputs)
    ])
    optimizer = Adam(learning_rate=lr)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

# ==============================================================================
# --- Final Model Training (Using Fixed Parameters - 3 LSTM Layers) ---
# ==============================================================================
print(f"\nBuilding and training FINAL model (FCM + Enhanced Fuzzy, 3 LSTM Layers) using FIXED parameters...")
final_n_features = final_train_feat_res.shape[2]; final_n_outputs = final_train_targ_res.shape[1]
K.clear_session()

# Use the fixed hyperparameters defined earlier
final_model = build_lstm_model(
    FIXED_LSTM_UNITS,
    FIXED_DENSE_UNITS,
    FIXED_DROPOUT,
    FIXED_LEARNING_RATE,
    final_n_features,
    final_n_outputs
)
print("Using Fixed Hyperparameters:")
print(f"  LSTM Units (x3 Layers): {FIXED_LSTM_UNITS}")
print(f"  Dense Units: {FIXED_DENSE_UNITS}")
print(f"  Dropout Rate: {FIXED_DROPOUT:.3f}")
print(f"  Learning Rate: {FIXED_LEARNING_RATE:.6f}")

final_model.summary()

# Define final training callbacks
final_early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1)
final_reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=8, min_lr=1e-7, verbose=1)

# Train the final model
history_final = final_model.fit(
    final_train_feat_res,
    final_train_targ_res,
    epochs=150, # Using 150 epochs as per previous request
    batch_size=32,
    validation_split=0.2,
    callbacks=[final_early_stopping, final_reduce_lr],
    verbose=1
)

# --- Plot Final Training Loss ---
plt.figure(figsize=(10, 6))
plt.plot(history_final.history['loss'], label='Training Loss')
plt.plot(history_final.history['val_loss'], label='Validation Loss')
plt.title(f'FINAL Model Loss (FCM + Enhanced Fuzzy, 3x{FIXED_LSTM_UNITS} LSTM, Fixed Params)')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
# Adjust ylim for better visualization
if len(history_final.history['loss']) > 5:
    max_loss_val = max(history_final.history['loss'][5:] + history_final.history['val_loss'][5:])
    min_loss_val = min(history_final.history['loss'] + history_final.history['val_loss'])
    plt.ylim(bottom=max(0, min_loss_val * 0.9), top=max_loss_val * 1.1)
elif len(history_final.history['loss']) > 0:
     max_loss_val = max(history_final.history['loss'] + history_final.history['val_loss'])
     min_loss_val = min(history_final.history['loss'] + history_final.history['val_loss'])
     plt.ylim(bottom=max(0, min_loss_val * 0.9), top=max_loss_val * 1.1)
else:
    plt.ylim(bottom=0)
plt.show()

if hasattr(final_early_stopping, 'stopped_epoch') and final_early_stopping.stopped_epoch > 0 and final_early_stopping.restore_best_weights:
    print(f"Final training stopped early at epoch {final_early_stopping.stopped_epoch + 1}. Best weights restored.")
elif hasattr(final_early_stopping, 'stopped_epoch') and final_early_stopping.stopped_epoch > 0:
     print(f"Final training stopped early at epoch {final_early_stopping.stopped_epoch + 1}, but best weights might not have been restored.")
else:
    print("Final training completed all epochs or early stopping did not trigger.")


# --- Final Evaluation ---
print(f"\nEvaluating FINAL model (FCM + Enhanced Fuzzy, 3x{FIXED_LSTM_UNITS} LSTM, Fixed Params) on Test Set (Days {list(TEST_DAYS)})...")
test_predictions_scaled = final_model.predict(final_test_feat_res)

# Inverse transform predictions and actuals
actual_test_targets = final_scaler_targ.inverse_transform(final_test_targ_res)
predicted_test_targets = final_scaler_targ.inverse_transform(test_predictions_scaled)
# Clip negative predictions
predicted_test_targets[predicted_test_targets < 0] = 0

# --- Create plotting dataframe ---
plot_ready = False
# Check length using the index of full_test_data and the length of predictions
if len(full_test_data.index) == len(actual_test_targets):
    # Create DataFrames for actual and predicted values using the test data index
    actual_df = pd.DataFrame(actual_test_targets, index=full_test_data.index, columns=zone_columns)
    pred_df = pd.DataFrame(predicted_test_targets, index=full_test_data.index, columns=[f"{zone}_predicted" for zone in zone_columns])

    # Combine index, actuals, and predictions into a new DataFrame for plotting
    plot_test_df = pd.concat([actual_df, pred_df], axis=1)
    plot_test_df['hour'] = plot_test_df.index.hour
    plot_test_df['day'] = plot_test_df.index.day # Use .day
    plot_test_df['month_name'] = plot_test_df.index.strftime('%B')
    plot_test_df['day_name'] = plot_test_df.index.day_name()

    plot_ready = True
    print(f"Plotting dataframe created successfully.")
else:
    print(f"Error: Length mismatch! Original Test DF index length {len(full_test_data.index)} vs Predictions length {len(actual_test_targets)}. Skipping plotting.")
    plot_test_df = None

# --- Plotting Results ---
if plot_ready and plot_test_df is not None:
    print(f"Plotting daily forecasts for FINAL model (FCM + Enhanced Fuzzy, 3x{FIXED_LSTM_UNITS} LSTM, Fixed Params) for days {list(TEST_DAYS)}...")
    unique_month_names = plot_test_df['month_name'].unique()

    for zone in zone_columns:
        zone_name_plot = zone.replace("_power_consumption", "").replace("zone_", "Zone ").capitalize()

        for month_name in unique_month_names:
            month_data = plot_test_df[plot_test_df['month_name'] == month_name]
            if month_data.empty: continue

            available_days_in_plot_df = month_data['day'].unique()
            plot_days = sorted([d for d in TEST_DAYS if d in available_days_in_plot_df])

            for day in plot_days:
                filtered_data = month_data[month_data['day'] == day]
                if filtered_data.empty: continue

                actual_col_name = zone
                predicted_col_name = f"{zone}_predicted"
                if actual_col_name not in filtered_data.columns or predicted_col_name not in filtered_data.columns:
                    print(f"Warning: Column missing for {zone}, {month_name}, {day}. Skipping plot.")
                    continue

                actual_day = filtered_data[actual_col_name]
                predicted_day = filtered_data[predicted_col_name]

                if not isinstance(actual_day, pd.Series) or not isinstance(predicted_day, pd.Series) or actual_day.ndim > 1 or predicted_day.ndim > 1:
                     print(f"Warning: Invalid data shape/type for metrics/plot for {zone}, {month_name}, {day}. Skipping.")
                     continue

                try:
                    daily_rmse = sqrt(mean_squared_error(actual_day, predicted_day))
                    daily_mae = mean_absolute_error(actual_day, predicted_day)
                    daily_r2 = r2_score(actual_day, predicted_day)
                except ValueError as e_met:
                    print(f"Warning: Metric calculation error for {zone}, {month_name}, {day}: {e_met}")
                    daily_rmse, daily_mae, daily_r2 = np.nan, np.nan, np.nan
                except Exception as e_gen:
                    print(f"Unexpected error calculating metrics for {zone}, {month_name}, {day}: {e_gen}")
                    daily_rmse, daily_mae, daily_r2 = np.nan, np.nan, np.nan


                plt.figure(figsize=(14, 7))
                bar_width = 0.35
                x = np.arange(len(filtered_data))
                plt.bar(x - bar_width/2, actual_day, width=bar_width, label='Actual', color='black', alpha=0.8)
                plt.bar(x + bar_width/2, predicted_day, width=bar_width, label='Predicted', color='orange', alpha=0.8)

                day_name_str = filtered_data['day_name'].iloc[0] if not filtered_data.empty else "UnknownDay"
                plt.title(f"{zone_name_plot} Load Forecast (FCM+Fuzzy, 3x{FIXED_LSTM_UNITS} LSTM) - {day_name_str}, {month_name} {day}", fontsize=14)
                plt.xlabel('Hour of Day', fontsize=12)
                plt.ylabel('Power Consumption', fontsize=12)
                plt.xticks(x, filtered_data['hour'], rotation=45, ha='right')
                plt.legend(fontsize=10)
                plt.grid(axis='y', linestyle='--', alpha=0.7)
                plt.tight_layout(rect=[0, 0, 0.85, 1]) # Adjust layout

                stats_text = (f"Daily Metrics:\n"
                              f"RMSE: {daily_rmse:.2f}\n"
                              f"MAE:  {daily_mae:.2f}\n"
                              f"R²:   {daily_r2:.3f}")
                plt.text(1.02, 0.95, stats_text, transform=plt.gca().transAxes, fontsize=10,
                         verticalalignment='top', bbox=dict(boxstyle='round,pad=0.5', facecolor='orange', alpha=0.6))
                plt.show()
else:
    print("Skipping plots due to data length mismatch or plotting dataframe creation failed.")


# --- Final Evaluation Metrics ---
print(f"\n--- FINAL Model (FCM + Enhanced Fuzzy, 3x{FIXED_LSTM_UNITS} LSTM, Fixed Params): Test Set Evaluation Metrics (Per Zone) ---")
all_actuals_flat = []
all_predictions_flat = []

for i, zone in enumerate(zone_columns):
    zone_name_print = zone.replace("zone_", "Zone_").replace("_power_consumption", "_")
    actual = actual_test_targets[:, i]
    predicted = predicted_test_targets[:, i]
    all_actuals_flat.extend(actual)
    all_predictions_flat.extend(predicted)

    rmse = sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    try:
        r2 = r2_score(actual, predicted)
    except ValueError:
        r2 = np.nan

    print(f"\n--- {zone_name_print} ---")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAE: {mae:.2f}")
    print(f"  R² Score: {r2:.4f}")


print(f"\n--- FINAL Model (FCM + Enhanced Fuzzy, 3x{FIXED_LSTM_UNITS} LSTM, Fixed Params): Overall Evaluation (All Zones Combined) ---")
all_actuals_flat = np.array(all_actuals_flat)
all_predictions_flat = np.array(all_predictions_flat)

overall_rmse = sqrt(mean_squared_error(all_actuals_flat, all_predictions_flat))
overall_mae = mean_absolute_error(all_actuals_flat, all_predictions_flat)
try:
    overall_r2 = r2_score(all_actuals_flat, all_predictions_flat)
except ValueError:
    overall_r2 = np.nan

# Calculate Tolerance Accuracy Safely
abs_error = np.abs(all_actuals_flat - all_predictions_flat)
non_zero_actuals = all_actuals_flat[all_actuals_flat > 1e-6]
mean_nonzero_actual = np.mean(non_zero_actuals) if len(non_zero_actuals) > 0 else 1.0

absolute_min_tolerance = 10
percentage_tolerance_value = (TOLERANCE_PERCENTAGE / 100.0) * np.abs(all_actuals_flat)
tolerance_threshold = np.maximum(percentage_tolerance_value, absolute_min_tolerance)

is_within_tolerance = abs_error <= tolerance_threshold
tolerance_accuracy = np.mean(is_within_tolerance) * 100

print(f"  Overall RMSE: {overall_rmse:.2f}")
print(f"  Overall MAE: {overall_mae:.2f}")
print(f"  Overall R² Score: {overall_r2:.4f}")
print(f"\n  Tolerance-Based Accuracy ({TOLERANCE_PERCENTAGE}%): {tolerance_accuracy:.2f}%")
print(f"     (Accuracy defined as % of predictions within +/- {TOLERANCE_PERCENTAGE}% of actual, with min absolute tolerance of {absolute_min_tolerance} units)")
print("----------------------------------------------------------")

print("\nProcessing complete.")

Output hidden; open in https://colab.research.google.com to view.